# Home Credit Default Risk

Can you predict how capable each applicant is of repaying a loan?

Many people struggle to get loans due to **insufficient or non-existent credit histories**. And, unfortunately, this population is often taken advantage of by untrustworthy lenders.

Home Credit strives to broaden financial inclusion for the **unbanked population by providing a positive and safe borrowing experience**. In order to make sure this underserved population has a positive loan experience, Home Credit makes use of a variety of alternative data--including telco and transactional information--to predict their clients' repayment abilities.

While Home Credit is currently using various statistical and machine learning methods to make these predictions, they're challenging Kagglers to help them unlock the full potential of their data. Doing so will ensure that clients capable of repayment are not rejected and that loans are given with a principal, maturity, and repayment calendar that will empower their clients to be successful.

**Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target.**

# Dataset

In [1]:
# #Python Libraries
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels
import pandas_profiling

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys
import time
import random
import requests
import datetime

import missingno as msno
import math
import sys
import gc
import os

# #sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestRegressor

# #sklearn - preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# #sklearn - metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score

# #XGBoost & LightGBM
import xgboost as xgb
import lightgbm as lgb

# #Missing value imputation
from fancyimpute import KNN, MICE

# #Hyperparameter Optimization
from hyperopt.pyll.base import scope
from hyperopt.pyll.stochastic import sample
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

pd.options.display.max_columns = 150

c:\users\karti\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Data Dictionary

In [2]:
!ls -l ../data/

total 2621364
-rw-r--r-- 1 karti 197609  26567651 May 17 18:06 application_test.csv
-rw-r--r-- 1 karti 197609 166133370 May 17 18:06 application_train.csv
-rw-r--r-- 1 karti 197609 170016717 May 17 18:08 bureau.csv
-rw-r--r-- 1 karti 197609 375592889 May 17 18:08 bureau_balance.csv
-rw-r--r-- 1 karti 197609 424582605 May 17 18:10 credit_card_balance.csv
-rw-r--r-- 1 karti 197609     37436 May 30 00:41 HomeCredit_columns_description.csv
-rw-r--r-- 1 karti 197609 723118349 May 17 18:13 installments_payments.csv
-rw-r--r-- 1 karti 197609 392703158 May 17 18:14 POS_CASH_balance.csv
-rw-r--r-- 1 karti 197609 404973293 May 17 18:15 previous_application.csv
-rw-r--r-- 1 karti 197609    536202 May 17 18:06 sample_submission.csv


- application_{train|test}.csv

This is the main table, broken into two files for Train (**with TARGET**) and Test (without TARGET).
Static data for all applications. **One row represents one loan in our data sample.**

Observations:
* Each row is unique

-----


- bureau.csv

All client's previous credits provided by other financial institutions that were reported to Credit Bureau (for clients who have a loan in our sample).
For every loan in our sample, there are as many rows as number of credits the client had in Credit Bureau before the application date.

- bureau_balance.csv

Monthly balances of previous credits in Credit Bureau.
This table has one row for each month of history of every previous credit reported to Credit Bureau – i.e the table has (#loans in sample * # of relative previous credits * # of months where we have some history observable for the previous credits) rows.

- POS_CASH_balance.csv

Monthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with Home Credit.
This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credits * # of months in which we have some history observable for the previous credits) rows.

- credit_card_balance.csv

Monthly balance snapshots of previous credit cards that the applicant has with Home Credit.
This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credit cards * # of months where we have some history observable for the previous credit card) rows.

-----

- previous_application.csv

All previous applications for Home Credit loans of clients who have loans in our sample.
There is one row for each previous application related to loans in our data sample.

-----

- installments_payments.csv

Repayment history for the previously disbursed credits in Home Credit related to the loans in our sample.
There is a) one row for every payment that was made plus b) one row each for missed payment.
One row is equivalent to one payment of one installment OR one installment corresponding to one payment of one previous Home Credit credit related to loans in our sample.

- HomeCredit_columns_description.csv

This file contains descriptions for the columns in the various data files.

![](https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png)

# Data Pre-processing

In [93]:
df_application_train_original = pd.read_csv("../data/application_train.csv")
df_application_test_original = pd.read_csv("../data/application_test.csv")
df_bureau_original = pd.read_csv("../data/bureau.csv")
df_bureau_balance_original = pd.read_csv("../data/bureau_balance.csv")
df_credit_card_balance_original = pd.read_csv("../data/credit_card_balance.csv")
df_installments_payments_original = pd.read_csv("../data/installments_payments.csv")
df_pos_cash_balance_original = pd.read_csv("../data/POS_CASH_balance.csv")
df_previous_application_original = pd.read_csv("../data/previous_application.csv")

In [94]:
df_application_train = pd.read_csv("../data/application_train.csv")
df_application_test = pd.read_csv("../data/application_test.csv")
df_bureau = pd.read_csv("../data/bureau.csv")
df_bureau_balance = pd.read_csv("../data/bureau_balance.csv")
df_credit_card_balance = pd.read_csv("../data/credit_card_balance.csv")
df_installments_payments = pd.read_csv("../data/installments_payments.csv")
df_pos_cash_balance = pd.read_csv("../data/POS_CASH_balance.csv")
df_previous_application = pd.read_csv("../data/previous_application.csv")

In [95]:
print("df_application_train: ", df_application_train.shape)
print("df_application_test: ", df_application_test.shape)
print("df_bureau: ", df_bureau.shape)
print("df_bureau_balance: ", df_bureau_balance.shape)
print("df_credit_card_balance: ", df_credit_card_balance.shape)
print("df_installments_payments: ", df_installments_payments.shape)
print("df_pos_cash_balance: ", df_pos_cash_balance.shape)
print("df_previous_application: ", df_previous_application.shape)

df_application_train:  (307511, 122)
df_application_test:  (48744, 121)
df_bureau:  (1716428, 17)
df_bureau_balance:  (27299925, 3)
df_credit_card_balance:  (3840312, 23)
df_installments_payments:  (13605401, 8)
df_pos_cash_balance:  (10001358, 8)
df_previous_application:  (1670214, 37)


In [96]:
gc.collect()

1408

## Feature: df_installments_payments

In [97]:
df_installments_payments['K_PREV_INSTALLMENT_PAYMENT_COUNT'] = df_installments_payments.groupby('SK_ID_CURR')['SK_ID_PREV'].transform('count')
df_installments_payments['K_NUM_INSTALMENT_NUMBER_SUM'] = df_installments_payments.groupby('SK_ID_CURR')['NUM_INSTALMENT_NUMBER'].transform(np.sum)

df_installments_payments['TEMP_DAYS_INSTALMENT'] = df_installments_payments.groupby('SK_ID_CURR')['DAYS_INSTALMENT'].transform(np.sum)
df_installments_payments['TEMP_DAYS_ENTRY_PAYMENT'] = df_installments_payments.groupby('SK_ID_CURR')['DAYS_ENTRY_PAYMENT'].transform(np.sum)
df_installments_payments['K_INST_DAYS_DIFF'] = df_installments_payments['TEMP_DAYS_INSTALMENT'] - df_installments_payments['TEMP_DAYS_ENTRY_PAYMENT']

df_installments_payments['TEMP_AMT_INSTALMENT'] = df_installments_payments.groupby('SK_ID_CURR')['AMT_INSTALMENT'].transform(np.sum)
df_installments_payments['TEMP_AMT_PAYMENT'] = df_installments_payments.groupby('SK_ID_CURR')['AMT_PAYMENT'].transform(np.sum)
df_installments_payments['K_INST_AMT_DIFF'] = df_installments_payments['TEMP_AMT_INSTALMENT'] - df_installments_payments['TEMP_AMT_PAYMENT']

# #Drop Duplicates
df_installments_payments = df_installments_payments[['SK_ID_CURR', 'K_PREV_INSTALLMENT_PAYMENT_COUNT', 'K_NUM_INSTALMENT_NUMBER_SUM', 'K_INST_DAYS_DIFF', 'K_INST_AMT_DIFF']].drop_duplicates()

In [98]:
# #CHECKPOINT
print("df_installments_payments", df_installments_payments.shape)
print(len(set(df_installments_payments["SK_ID_CURR"]).intersection(set(df_application_train["SK_ID_CURR"]))))
print(len(set(df_installments_payments["SK_ID_CURR"]).intersection(set(df_application_test["SK_ID_CURR"]))))
print("Sum: ", 291643 + 47944)

df_installments_payments (339587, 5)
291643
47944
Sum:  339587


## Feature: df_credit_card_balance

In [99]:
df_credit_card_balance['K_PREV_CREDIT_CARD_BALANCE_COUNT'] = df_credit_card_balance.groupby('SK_ID_CURR')['SK_ID_PREV'].transform('count')

df_credit_card_balance['K_MONTHS_BALANCE_MAX'] = df_credit_card_balance.groupby('SK_ID_CURR')['MONTHS_BALANCE'].transform(np.max)
df_credit_card_balance['K_MONTHS_BALANCE_MIN'] = df_credit_card_balance.groupby('SK_ID_CURR')['MONTHS_BALANCE'].transform(np.min)

df_credit_card_balance['TEMP_AMT_BALANCE'] = df_credit_card_balance.groupby('SK_ID_CURR')['AMT_BALANCE'].transform(lambda x:x+1)
df_credit_card_balance['TEMP_AMT_CREDIT_LIMIT_ACTUAL'] = df_credit_card_balance.groupby('SK_ID_CURR')['AMT_CREDIT_LIMIT_ACTUAL'].transform(lambda x:x+1)
df_credit_card_balance['TEMP_UTILIZATION'] = df_credit_card_balance['TEMP_AMT_BALANCE']/df_credit_card_balance['TEMP_AMT_CREDIT_LIMIT_ACTUAL']
df_credit_card_balance['K_CREDIT_UTILIZATION_MEAN'] = df_credit_card_balance.groupby('SK_ID_CURR')['TEMP_UTILIZATION'].transform(np.mean)
df_credit_card_balance['K_CREDIT_UTILIZATION_MIN'] = df_credit_card_balance.groupby('SK_ID_CURR')['TEMP_UTILIZATION'].transform(np.min)
df_credit_card_balance['K_CREDIT_UTILIZATION_MAX'] = df_credit_card_balance.groupby('SK_ID_CURR')['TEMP_UTILIZATION'].transform(np.max)

# #Validation: SK_ID_CURR = 105755
# #AMT_DRAWINGS_CURRENT = AMT_DRAWINGS_ATM_CURRENT + AMT_DRAWINGS_OTHER_CURRENT + AMT_DRAWINGS_POS_CURRENT
df_credit_card_balance['K_AMT_DRAWINGS_CURRENT_MEAN'] = df_credit_card_balance.groupby('SK_ID_CURR')['AMT_DRAWINGS_CURRENT'].transform(np.mean)
df_credit_card_balance['K_AMT_DRAWINGS_CURRENT_MIN'] = df_credit_card_balance.groupby('SK_ID_CURR')['AMT_DRAWINGS_CURRENT'].transform(np.min)
df_credit_card_balance['K_AMT_DRAWINGS_CURRENT_MAX'] = df_credit_card_balance.groupby('SK_ID_CURR')['AMT_DRAWINGS_CURRENT'].transform(np.max)

df_credit_card_balance['TEMP_AMT_PAYMENT_TOTAL_CURRENT'] = df_credit_card_balance.groupby('SK_ID_CURR')['AMT_PAYMENT_TOTAL_CURRENT'].transform(lambda x:x+1)
df_credit_card_balance['TEMP_AMT_TOTAL_RECEIVABLE'] = df_credit_card_balance.groupby('SK_ID_CURR')['AMT_TOTAL_RECEIVABLE'].transform(lambda x:x+1)
df_credit_card_balance['TEMP_AMT_PAYMENT_OVER_RECEIVABLE'] = df_credit_card_balance['TEMP_AMT_PAYMENT_TOTAL_CURRENT']/df_credit_card_balance['TEMP_AMT_TOTAL_RECEIVABLE']
df_credit_card_balance['K_AMT_PAYMENT_OVER_RECEIVABLE_MEAN'] = df_credit_card_balance.groupby('SK_ID_CURR')['TEMP_AMT_PAYMENT_OVER_RECEIVABLE'].transform(np.mean)
df_credit_card_balance['K_AMT_PAYMENT_OVER_RECEIVABLE_MIN'] = df_credit_card_balance.groupby('SK_ID_CURR')['TEMP_AMT_PAYMENT_OVER_RECEIVABLE'].transform(np.min)
df_credit_card_balance['K_AMT_PAYMENT_OVER_RECEIVABLE_MAX'] = df_credit_card_balance.groupby('SK_ID_CURR')['TEMP_AMT_PAYMENT_OVER_RECEIVABLE'].transform(np.max)

# #CNT_DRAWINGS_CURRENT = CNT_DRAWINGS_ATM_CURRENT + CNT_DRAWINGS_OTHER_CURRENT + CNT_DRAWINGS_POS_CURRENT
df_credit_card_balance['K_CNT_DRAWINGS_CURRENT_MEAN'] = df_credit_card_balance.groupby('SK_ID_CURR')['CNT_DRAWINGS_CURRENT'].transform(np.mean)
df_credit_card_balance['K_CNT_DRAWINGS_CURRENT_MIN'] = df_credit_card_balance.groupby('SK_ID_CURR')['CNT_DRAWINGS_CURRENT'].transform(np.min)
df_credit_card_balance['K_CNT_DRAWINGS_CURRENT_MAX'] = df_credit_card_balance.groupby('SK_ID_CURR')['CNT_DRAWINGS_CURRENT'].transform(np.max)


# #Drop Duplicates
df_credit_card_balance = df_credit_card_balance[['SK_ID_CURR', 'K_PREV_CREDIT_CARD_BALANCE_COUNT', 'K_MONTHS_BALANCE_MAX', 'K_MONTHS_BALANCE_MIN', 'K_CREDIT_UTILIZATION_MEAN', 'K_CREDIT_UTILIZATION_MIN', 'K_CREDIT_UTILIZATION_MAX', 'K_AMT_DRAWINGS_CURRENT_MEAN', 'K_AMT_DRAWINGS_CURRENT_MIN', 'K_AMT_DRAWINGS_CURRENT_MAX', 'K_AMT_PAYMENT_OVER_RECEIVABLE_MEAN', 'K_AMT_PAYMENT_OVER_RECEIVABLE_MIN', 'K_AMT_PAYMENT_OVER_RECEIVABLE_MAX', 'K_CNT_DRAWINGS_CURRENT_MEAN', 'K_CNT_DRAWINGS_CURRENT_MIN', 'K_CNT_DRAWINGS_CURRENT_MAX']].drop_duplicates()

In [100]:
# #CHECKPOINT
print("df_credit_card_balance", df_credit_card_balance.shape)
print(len(set(df_credit_card_balance["SK_ID_CURR"]).intersection(set(df_application_train["SK_ID_CURR"]))))
print(len(set(df_credit_card_balance["SK_ID_CURR"]).intersection(set(df_application_test["SK_ID_CURR"]))))
print("Sum: ", 86905 + 16653)

df_credit_card_balance (103558, 16)
86905
16653
Sum:  103558


## Feature: df_pos_cash_balance

In [101]:
df_pos_cash_balance['K_PREV_POS_CASH_BALANCE_COUNT'] = df_pos_cash_balance.groupby('SK_ID_CURR')['SK_ID_PREV'].transform('count')

df_pos_cash_balance['K_MONTHS_BALANCE_POS_CASH_MEAN'] = df_pos_cash_balance.groupby('SK_ID_CURR')['MONTHS_BALANCE'].transform(np.mean)
df_pos_cash_balance['K_MONTHS_BALANCE_POS_CASH_MAX'] = df_pos_cash_balance.groupby('SK_ID_CURR')['MONTHS_BALANCE'].transform(np.max)
df_pos_cash_balance['K_MONTHS_BALANCE_POS_CASH_MIN'] = df_pos_cash_balance.groupby('SK_ID_CURR')['MONTHS_BALANCE'].transform(np.min)

df_pos_cash_balance['K_CNT_INSTALMENT_MEAN'] = df_pos_cash_balance.groupby('SK_ID_CURR')['CNT_INSTALMENT'].transform(np.mean)
df_pos_cash_balance['K_CNT_INSTALMENT_MAX'] = df_pos_cash_balance.groupby('SK_ID_CURR')['CNT_INSTALMENT'].transform(np.max)
df_pos_cash_balance['K_CNT_INSTALMENT_MIN'] = df_pos_cash_balance.groupby('SK_ID_CURR')['CNT_INSTALMENT'].transform(np.min)

df_pos_cash_balance['K_CNT_INSTALMENT_FUTURE_MEAN'] = df_pos_cash_balance.groupby('SK_ID_CURR')['CNT_INSTALMENT_FUTURE'].transform(np.mean)
df_pos_cash_balance['K_CNT_INSTALMENT_FUTURE_MAX'] = df_pos_cash_balance.groupby('SK_ID_CURR')['CNT_INSTALMENT_FUTURE'].transform(np.max)
df_pos_cash_balance['K_CNT_INSTALMENT_FUTURE_MIN'] = df_pos_cash_balance.groupby('SK_ID_CURR')['CNT_INSTALMENT_FUTURE'].transform(np.min)

# #Drop Duplicates
df_pos_cash_balance = df_pos_cash_balance[['SK_ID_CURR', 'K_PREV_POS_CASH_BALANCE_COUNT', 
                                           'K_MONTHS_BALANCE_POS_CASH_MEAN','K_MONTHS_BALANCE_POS_CASH_MAX', 'K_MONTHS_BALANCE_POS_CASH_MIN', 
                                           'K_CNT_INSTALMENT_MEAN', 'K_CNT_INSTALMENT_MAX', 'K_CNT_INSTALMENT_MIN', 
                                           'K_CNT_INSTALMENT_FUTURE_MEAN', 'K_CNT_INSTALMENT_FUTURE_MAX', 'K_CNT_INSTALMENT_FUTURE_MIN']].drop_duplicates()

## Feature: df_previous_application

In [102]:
df_previous_application['K_PREV_PREVIOUS_APPLICATION_COUNT'] = df_previous_application.groupby('SK_ID_CURR')['SK_ID_PREV'].transform('count')

df_previous_application['K_AMT_ANNUITY_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['AMT_ANNUITY'].transform(np.mean)
df_previous_application['K_AMT_ANNUITY_MAX'] = df_previous_application.groupby('SK_ID_CURR')['AMT_ANNUITY'].transform(np.max)
df_previous_application['K_AMT_ANNUITY_MIN'] = df_previous_application.groupby('SK_ID_CURR')['AMT_ANNUITY'].transform(np.min)

df_previous_application['TEMP_CREDIT_ALLOCATED'] = df_previous_application['AMT_CREDIT']/df_previous_application['AMT_APPLICATION']
df_previous_application['K_CREDIT_ALLOCATED_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['TEMP_CREDIT_ALLOCATED'].transform(np.mean)
df_previous_application['K_CREDIT_ALLOCATED_MAX'] = df_previous_application.groupby('SK_ID_CURR')['TEMP_CREDIT_ALLOCATED'].transform(np.max)
df_previous_application['K_CREDIT_ALLOCATED_MIN'] = df_previous_application.groupby('SK_ID_CURR')['TEMP_CREDIT_ALLOCATED'].transform(np.min)

df_previous_application['K_AMT_DOWN_PAYMENT_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['AMT_DOWN_PAYMENT'].transform(np.mean)
df_previous_application['K_AMT_DOWN_PAYMENT_MAX'] = df_previous_application.groupby('SK_ID_CURR')['AMT_DOWN_PAYMENT'].transform(np.max)
df_previous_application['K_AMT_DOWN_PAYMENT_MIN'] = df_previous_application.groupby('SK_ID_CURR')['AMT_DOWN_PAYMENT'].transform(np.min)

df_previous_application['K_AMT_GOODS_PRICE_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['AMT_GOODS_PRICE'].transform(np.mean)
df_previous_application['K_AMT_GOODS_PRICE_MAX'] = df_previous_application.groupby('SK_ID_CURR')['AMT_GOODS_PRICE'].transform(np.max)
df_previous_application['K_AMT_GOODS_PRICE_MIN'] = df_previous_application.groupby('SK_ID_CURR')['AMT_GOODS_PRICE'].transform(np.min)

df_previous_application['K_DAYS_DECISION_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_DECISION'].transform(np.mean)
df_previous_application['K_DAYS_DECISION_MAX'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_DECISION'].transform(np.max)
df_previous_application['K_DAYS_DECISION_MIN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_DECISION'].transform(np.min)

df_previous_application['K_CNT_PAYMENT_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['CNT_PAYMENT'].transform(np.mean)
df_previous_application['K_CNT_PAYMENT_MAX'] = df_previous_application.groupby('SK_ID_CURR')['CNT_PAYMENT'].transform(np.max)
df_previous_application['K_CNT_PAYMENT_MIN'] = df_previous_application.groupby('SK_ID_CURR')['CNT_PAYMENT'].transform(np.min)

df_previous_application['K_DAYS_FIRST_DRAWING_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_FIRST_DRAWING'].transform(np.mean)
df_previous_application['K_DAYS_FIRST_DRAWING_MAX'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_FIRST_DRAWING'].transform(np.max)
df_previous_application['K_DAYS_FIRST_DRAWING_MIN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_FIRST_DRAWING'].transform(np.min)

df_previous_application['K_DAYS_FIRST_DUE_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_FIRST_DUE'].transform(np.mean)
df_previous_application['K_DAYS_FIRST_DUE_MAX'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_FIRST_DUE'].transform(np.max)
df_previous_application['K_DAYS_FIRST_DUE_MIN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_FIRST_DUE'].transform(np.min)

df_previous_application['K_DAYS_LAST_DUE_1ST_VERSION_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_LAST_DUE_1ST_VERSION'].transform(np.mean)
df_previous_application['K_DAYS_LAST_DUE_1ST_VERSION_MAX'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_LAST_DUE_1ST_VERSION'].transform(np.max)
df_previous_application['K_DAYS_LAST_DUE_1ST_VERSION_MIN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_LAST_DUE_1ST_VERSION'].transform(np.min)

df_previous_application['K_DAYS_LAST_DUE_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_LAST_DUE'].transform(np.mean)
df_previous_application['K_DAYS_LAST_DUE_MAX'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_LAST_DUE'].transform(np.max)
df_previous_application['K_DAYS_LAST_DUE_MIN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_LAST_DUE'].transform(np.min)

df_previous_application['K_DAYS_TERMINATION_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_TERMINATION'].transform(np.mean)
df_previous_application['K_DAYS_TERMINATION_MAX'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_TERMINATION'].transform(np.max)
df_previous_application['K_DAYS_TERMINATION_MIN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_TERMINATION'].transform(np.min)


# #Drop Duplicates
df_previous_application = df_previous_application[['SK_ID_CURR', 'K_PREV_PREVIOUS_APPLICATION_COUNT', 
                                          'K_AMT_ANNUITY_MEAN', 'K_AMT_ANNUITY_MAX', 'K_AMT_ANNUITY_MIN',
                                          'K_CREDIT_ALLOCATED_MEAN', 'K_CREDIT_ALLOCATED_MAX', 'K_CREDIT_ALLOCATED_MIN',
                                          'K_AMT_DOWN_PAYMENT_MEAN', 'K_AMT_DOWN_PAYMENT_MAX', 'K_AMT_DOWN_PAYMENT_MIN',
                                          'K_AMT_GOODS_PRICE_MEAN', 'K_AMT_GOODS_PRICE_MAX', 'K_AMT_GOODS_PRICE_MIN',
                                          'K_DAYS_DECISION_MEAN', 'K_DAYS_DECISION_MAX', 'K_DAYS_DECISION_MIN',
                                          'K_CNT_PAYMENT_MEAN', 'K_CNT_PAYMENT_MAX', 'K_CNT_PAYMENT_MIN',
                                          'K_DAYS_FIRST_DRAWING_MEAN', 'K_DAYS_FIRST_DRAWING_MAX', 'K_DAYS_FIRST_DRAWING_MIN',
                                          'K_DAYS_FIRST_DUE_MEAN', 'K_DAYS_FIRST_DUE_MAX', 'K_DAYS_FIRST_DUE_MIN',
                                          'K_DAYS_LAST_DUE_1ST_VERSION_MEAN', 'K_DAYS_LAST_DUE_1ST_VERSION_MAX', 'K_DAYS_LAST_DUE_1ST_VERSION_MIN',
                                          'K_DAYS_LAST_DUE_MEAN', 'K_DAYS_LAST_DUE_MAX', 'K_DAYS_LAST_DUE_MIN',
                                          'K_DAYS_TERMINATION_MEAN', 'K_DAYS_TERMINATION_MAX', 'K_DAYS_TERMINATION_MIN']].drop_duplicates()

In [103]:
# df_bureau = pd.read_csv("../data/bureau.csv")
# df_bureau_balance = pd.read_csv("../data/bureau_balance.csv")

## Feature: df_bureau_balance

In [104]:
# df_bureau_balance['K_BUREAU_BALANCE_COUNT'] = df_bureau_balance.groupby('SK_ID_BUREAU')['SK_ID_BUREAU'].transform('count')

df_bureau_balance['K_BUREAU_BALANCE_MONTHS_BALANCE_MEAN'] = df_bureau_balance.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].transform(np.mean)
df_bureau_balance['K_BUREAU_BALANCE_MONTHS_BALANCE_MAX'] = df_bureau_balance.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].transform(np.max)
df_bureau_balance['K_BUREAU_BALANCE_MONTHS_BALANCE_MIN'] = df_bureau_balance.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].transform(np.min)

# #Drop Duplicates
df_bureau_balance = df_bureau_balance[['SK_ID_BUREAU','K_BUREAU_BALANCE_MONTHS_BALANCE_MEAN',
                                                 'K_BUREAU_BALANCE_MONTHS_BALANCE_MAX',
                                                'K_BUREAU_BALANCE_MONTHS_BALANCE_MIN']].drop_duplicates()

In [105]:
len(df_bureau_balance['SK_ID_BUREAU'].unique())

817395

## Feature: df_bureau

In [106]:
len(df_bureau["SK_ID_BUREAU"].unique())

1716428

In [107]:
df_bureau.shape

(1716428, 17)

In [108]:
df_bureau = pd.merge(df_bureau, df_bureau_balance, on="SK_ID_BUREAU", how="left", suffixes=('_bureau', '_bureau_balance'))

In [109]:
df_bureau.shape

(1716428, 20)

In [110]:
# #Feature - SK_ID_BUREAU represents each loan application.
# #Grouping by SK_ID_CURR significes the number of previous loans per applicant.
df_bureau['K_BUREAU_COUNT'] = df_bureau.groupby('SK_ID_CURR')['SK_ID_BUREAU'].transform('count')

# # #Feature - CREDIT_ACTIVE
# # #Frequency Encoding
# temp_bureau_credit_active = df_bureau.groupby(['SK_ID_CURR','CREDIT_ACTIVE']).size()/df_bureau.groupby(['SK_ID_CURR']).size()
# temp_bureau_credit_active = temp_bureau_credit_active.to_frame().reset_index().rename(columns= {0: 'TEMP_BUREAU_CREDIT_ACTIVE_FREQENCODE'})
# temp_bureau_credit_active = temp_bureau_credit_active.pivot(index='SK_ID_CURR', columns='CREDIT_ACTIVE', values='TEMP_BUREAU_CREDIT_ACTIVE_FREQENCODE')
# temp_bureau_credit_active.reset_index(inplace = True)
# temp_bureau_credit_active.columns = ['SK_ID_CURR', 'K_CREDIT_ACTIVE_ACTIVE', 'K_CREDIT_ACTIVE_BADDEBT', 'K_CREDIT_ACTIVE_CLOSED', 'K_CREDIT_ACTIVE_SOLD']
# df_bureau = pd.merge(df_bureau, temp_bureau_credit_active, on=["SK_ID_CURR"], how="left", suffixes=('_bureau', '_credit_active_percentage'))
# del temp_bureau_credit_active

# # #Feature - CREDIT_CURRENCY
# temp_credit_currency = df_bureau.groupby('SK_ID_CURR')['CREDIT_CURRENCY'].agg(lambda x:x.value_counts().index[0])
# df_bureau['K_BUREAU_CREDIT_CURRENCY_MOSTCOMMON_CATEGORY'] = temp_credit_currency.to_frame()['CREDIT_CURRENCY'].astype('category').cat.codes
# del temp_credit_currency

# # #Feature - DAYS_CREDIT
df_bureau['K_BUREAU_DAYS_CREDIT_MEAN'] = df_bureau.groupby('SK_ID_CURR')['DAYS_CREDIT'].transform(np.mean)
df_bureau['K_BUREAU_DAYS_CREDIT_MAX'] = df_bureau.groupby('SK_ID_CURR')['DAYS_CREDIT'].transform(np.max)
df_bureau['K_BUREAU_DAYS_CREDIT_MIN'] = df_bureau.groupby('SK_ID_CURR')['DAYS_CREDIT'].transform(np.min)
gc.collect()
# # #Successive difference between credit application per customer - Mean, Min, Max
temp_bureau_days_credit = df_bureau.copy()
temp_bureau_days_credit.sort_values(['SK_ID_CURR', 'DAYS_CREDIT'], inplace=True)
temp_bureau_days_credit['temp_successive_diff'] = temp_bureau_days_credit.groupby('SK_ID_CURR')['DAYS_CREDIT'].transform(lambda ele: ele.diff())
temp_bureau_days_credit['K_BUREAU_DAYS_CREDIT_SORTED_SUCCESSIVE_DIFF_MEAN'] = temp_bureau_days_credit.groupby('SK_ID_CURR')['temp_successive_diff'].transform(np.mean)
df_bureau = pd.merge(df_bureau, temp_bureau_days_credit[['SK_ID_CURR','K_BUREAU_DAYS_CREDIT_SORTED_SUCCESSIVE_DIFF_MEAN']].drop_duplicates(), 
                     on="SK_ID_CURR", how="left", suffixes=('_bureau', '_days_credit_sorted_successive_diff'))
# del temp_bureau_days_credit

df_bureau['K_BUREAU_CREDIT_DAY_OVERDUE_MEAN'] = df_bureau.groupby('SK_ID_CURR')['CREDIT_DAY_OVERDUE'].transform(np.mean)
df_bureau['K_BUREAU_CREDIT_DAY_OVERDUE_MAX'] = df_bureau.groupby('SK_ID_CURR')['CREDIT_DAY_OVERDUE'].transform(np.max)
df_bureau['K_BUREAU_CREDIT_DAY_OVERDUE_MIN'] = df_bureau.groupby('SK_ID_CURR')['CREDIT_DAY_OVERDUE'].transform(np.min)

df_bureau['K_BUREAU_DAYS_CREDIT_ENDDATE_MEAN'] = df_bureau.groupby('SK_ID_CURR')['DAYS_CREDIT_ENDDATE'].transform(np.mean)
df_bureau['K_BUREAU_DAYS_CREDIT_ENDDATE_MAX'] = df_bureau.groupby('SK_ID_CURR')['DAYS_CREDIT_ENDDATE'].transform(np.max)
df_bureau['K_BUREAU_DAYS_CREDIT_ENDDATE_MIN'] = df_bureau.groupby('SK_ID_CURR')['DAYS_CREDIT_ENDDATE'].transform(np.min)

df_bureau['K_BUREAU_DAYS_ENDDATE_FACT_MEAN'] = df_bureau.groupby('SK_ID_CURR')['DAYS_ENDDATE_FACT'].transform(np.mean)
df_bureau['K_BUREAU_DAYS_ENDDATE_FACT_MAX'] = df_bureau.groupby('SK_ID_CURR')['DAYS_ENDDATE_FACT'].transform(np.max)
df_bureau['K_BUREAU_DAYS_ENDDATE_FACT_MIN'] = df_bureau.groupby('SK_ID_CURR')['DAYS_ENDDATE_FACT'].transform(np.min)

df_bureau['K_BUREAU_AMT_CREDIT_MAX_OVERDUE_MEAN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_MAX_OVERDUE'].transform(np.mean)
df_bureau['K_BUREAU_AMT_CREDIT_MAX_OVERDUE_MAX'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_MAX_OVERDUE'].transform(np.max)
df_bureau['K_BUREAU_AMT_CREDIT_MAX_OVERDUE_MIN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_MAX_OVERDUE'].transform(np.min)


df_bureau['K_BUREAU_CNT_CREDIT_PROLONG_MAX'] = df_bureau.groupby('SK_ID_CURR')['CNT_CREDIT_PROLONG'].transform(np.max)

# #To-Do: Calculate a utilization metric for some of the features below?
df_bureau['K_BUREAU_AMT_CREDIT_SUM_MEAN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM'].transform(np.mean)
df_bureau['K_BUREAU_AMT_CREDIT_SUM_MAX'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM'].transform(np.max)
df_bureau['K_BUREAU_AMT_CREDIT_SUM_MIN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM'].transform(np.min)

df_bureau['K_BUREAU_AMT_CREDIT_SUM_DEBT_MEAN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_DEBT'].transform(np.mean)
df_bureau['K_BUREAU_AMT_CREDIT_SUM_DEBT_MAX'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_DEBT'].transform(np.max)
df_bureau['K_BUREAU_AMT_CREDIT_SUM_DEBT_MIN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_DEBT'].transform(np.min)

df_bureau['K_BUREAU_AMT_CREDIT_SUM_LIMIT_MEAN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_LIMIT'].transform(np.mean)
df_bureau['K_BUREAU_AMT_CREDIT_SUM_LIMIT_MAX'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_LIMIT'].transform(np.max)
df_bureau['K_BUREAU_AMT_CREDIT_SUM_LIMIT_MIN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_LIMIT'].transform(np.min)

df_bureau['K_BUREAU_AMT_CREDIT_SUM_OVERDUE_MEAN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_OVERDUE'].transform(np.mean)
df_bureau['K_BUREAU_AMT_CREDIT_SUM_OVERDUE_MAX'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_OVERDUE'].transform(np.max)
df_bureau['K_BUREAU_AMT_CREDIT_SUM_OVERDUE_MIN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_OVERDUE'].transform(np.min)

df_bureau['K_BUREAU_AMT_ANNUITY_MEAN'] = df_bureau.groupby('SK_ID_CURR')['AMT_ANNUITY'].transform(np.mean)
df_bureau['K_BUREAU_AMT_ANNUITY_MAX'] = df_bureau.groupby('SK_ID_CURR')['AMT_ANNUITY'].transform(np.max)
df_bureau['K_BUREAU_AMT_ANNUITY_MIN'] = df_bureau.groupby('SK_ID_CURR')['AMT_ANNUITY'].transform(np.min)

# #Added from df_bureau_balance
df_bureau['K_BUREAU_BALANCE_MONTHS_BALANCE_MEAN'] = df_bureau.groupby('SK_ID_CURR')['K_BUREAU_BALANCE_MONTHS_BALANCE_MEAN'].transform(np.mean)
df_bureau['K_BUREAU_BALANCE_MONTHS_BALANCE_MAX'] = df_bureau.groupby('SK_ID_CURR')['K_BUREAU_BALANCE_MONTHS_BALANCE_MAX'].transform(np.max)
df_bureau['K_BUREAU_BALANCE_MONTHS_BALANCE_MIN'] = df_bureau.groupby('SK_ID_CURR')['K_BUREAU_BALANCE_MONTHS_BALANCE_MIN'].transform(np.min)


#Drop Duplicates
df_bureau = df_bureau[['SK_ID_CURR', 'K_BUREAU_COUNT',
                       'K_BUREAU_DAYS_CREDIT_MEAN', 'K_BUREAU_DAYS_CREDIT_MAX', 'K_BUREAU_DAYS_CREDIT_MIN',
                       'K_BUREAU_DAYS_CREDIT_SORTED_SUCCESSIVE_DIFF_MEAN',
                       'K_BUREAU_CREDIT_DAY_OVERDUE_MEAN', 'K_BUREAU_CREDIT_DAY_OVERDUE_MAX', 'K_BUREAU_CREDIT_DAY_OVERDUE_MIN',
                      'K_BUREAU_DAYS_CREDIT_ENDDATE_MEAN', 'K_BUREAU_DAYS_CREDIT_ENDDATE_MAX', 'K_BUREAU_DAYS_CREDIT_ENDDATE_MIN',
                      'K_BUREAU_DAYS_ENDDATE_FACT_MEAN', 'K_BUREAU_DAYS_ENDDATE_FACT_MAX', 'K_BUREAU_DAYS_ENDDATE_FACT_MIN',
                      'K_BUREAU_AMT_CREDIT_MAX_OVERDUE_MEAN', 'K_BUREAU_AMT_CREDIT_MAX_OVERDUE_MAX', 'K_BUREAU_AMT_CREDIT_MAX_OVERDUE_MIN',
                      'K_BUREAU_CNT_CREDIT_PROLONG_MAX',
                      'K_BUREAU_AMT_CREDIT_SUM_MEAN', 'K_BUREAU_AMT_CREDIT_SUM_MAX', 'K_BUREAU_AMT_CREDIT_SUM_MIN',
                      'K_BUREAU_AMT_CREDIT_SUM_DEBT_MEAN', 'K_BUREAU_AMT_CREDIT_SUM_DEBT_MAX', 'K_BUREAU_AMT_CREDIT_SUM_DEBT_MIN',
                      'K_BUREAU_AMT_CREDIT_SUM_LIMIT_MEAN', 'K_BUREAU_AMT_CREDIT_SUM_LIMIT_MAX', 'K_BUREAU_AMT_CREDIT_SUM_LIMIT_MIN',
                      'K_BUREAU_AMT_CREDIT_SUM_OVERDUE_MEAN', 'K_BUREAU_AMT_CREDIT_SUM_OVERDUE_MAX', 'K_BUREAU_AMT_CREDIT_SUM_OVERDUE_MIN',
                      'K_BUREAU_AMT_ANNUITY_MEAN', 'K_BUREAU_AMT_ANNUITY_MAX', 'K_BUREAU_AMT_ANNUITY_MIN',
                      'K_BUREAU_BALANCE_MONTHS_BALANCE_MEAN', 'K_BUREAU_BALANCE_MONTHS_BALANCE_MAX', 'K_BUREAU_BALANCE_MONTHS_BALANCE_MIN']].drop_duplicates()

In [111]:
df_bureau.shape

(362366, 38)

In [112]:
# #CHECKPOINT
print("df_bureau_original", df_bureau_original.shape)
print(len(set(df_bureau_original["SK_ID_CURR"]).intersection(set(df_application_train["SK_ID_CURR"]))))
print(len(set(df_bureau_original["SK_ID_CURR"]).intersection(set(df_application_test["SK_ID_CURR"]))))
print("Sum: ", 263491 + 42320)

df_bureau_original (1716428, 17)
263491
42320
Sum:  305811


In [113]:
# #CHECKPOINT
print("df_bureau", df_bureau.shape)
print(len(set(df_bureau["SK_ID_CURR"]).intersection(set(df_application_train["SK_ID_CURR"]))))
print(len(set(df_bureau["SK_ID_CURR"]).intersection(set(df_application_test["SK_ID_CURR"]))))
print("Sum: ", 263491 + 42320)

df_bureau (362366, 38)
263491
42320
Sum:  305811


In [114]:
gc.collect()

43

## Feature MAIN TABLE: df_application_train

In [115]:
# #Feature - Divide existing features
df_application_train['K_APP_CREDIT_TO_INCOME_RATIO'] = df_application_train['AMT_CREDIT']/df_application_train['AMT_INCOME_TOTAL']
df_application_train['K_APP_ANNUITY_TO_INCOME_RATIO'] = df_application_train['AMT_ANNUITY']/df_application_train['AMT_INCOME_TOTAL']
df_application_train['K_APP_ANNUITY_TO_CREDIT_RATIO'] = df_application_train['AMT_ANNUITY']/df_application_train['AMT_CREDIT']

df_application_train['K_APP_GOODSPRICE_TO_CREDIT_RATIO'] = df_application_train['AMT_GOODS_PRICE']/df_application_train['AMT_CREDIT']

# #Feature - Income, Education, Family Status
df_application_train['K_APP_INCOME_EDUCATION'] = df_application_train['NAME_INCOME_TYPE'] + df_application_train['NAME_EDUCATION_TYPE']
df_application_train['K_APP_INCOME_EDUCATION_FAMILY'] = df_application_train['NAME_INCOME_TYPE'] + df_application_train['NAME_EDUCATION_TYPE'] + df_application_train['NAME_FAMILY_STATUS']

# #Feature - Age in years
df_application_train['K_DAYS_BIRTH_TO_EMPLOYED_RATIO'] = df_application_train['DAYS_BIRTH']/df_application_train['DAYS_EMPLOYED']

In [116]:
# #Feature - Divide existing features
df_application_test['K_APP_CREDIT_TO_INCOME_RATIO'] = df_application_test['AMT_CREDIT']/df_application_test['AMT_INCOME_TOTAL']
df_application_test['K_APP_ANNUITY_TO_INCOME_RATIO'] = df_application_test['AMT_ANNUITY']/df_application_test['AMT_INCOME_TOTAL']
df_application_test['K_APP_ANNUITY_TO_CREDIT_RATIO'] = df_application_test['AMT_ANNUITY']/df_application_test['AMT_CREDIT']

df_application_test['K_APP_GOODSPRICE_TO_CREDIT_RATIO'] = df_application_test['AMT_GOODS_PRICE']/df_application_test['AMT_CREDIT']

# #Feature - Income, Education, Family Status
df_application_test['K_APP_INCOME_EDUCATION'] = df_application_test['NAME_INCOME_TYPE'] + df_application_test['NAME_EDUCATION_TYPE']
df_application_test['K_APP_INCOME_EDUCATION_FAMILY'] = df_application_test['NAME_INCOME_TYPE'] + df_application_test['NAME_EDUCATION_TYPE'] + df_application_test['NAME_FAMILY_STATUS']

# #Feature - Age in years
df_application_test['K_DAYS_BIRTH_TO_EMPLOYED_RATIO'] = df_application_test['DAYS_BIRTH']/df_application_test['DAYS_EMPLOYED']

In [117]:
gc.collect()

21

# Combine Datasets

## Encode categorical columns

In [118]:
arr_categorical_columns = df_application_train.select_dtypes(['object']).columns
for var_col in arr_categorical_columns:
    df_application_train[var_col] = df_application_train[var_col].astype('category').cat.codes
gc.collect()

arr_categorical_columns = df_application_test.select_dtypes(['object']).columns
for var_col in arr_categorical_columns:
    df_application_test[var_col] = df_application_test[var_col].astype('category').cat.codes
gc.collect()

# arr_categorical_columns = df_credit_card_balance.select_dtypes(['object']).columns
# for var_col in arr_categorical_columns:
#     df_credit_card_balance[var_col] = df_credit_card_balance[var_col].astype('category').cat.codes

126

## Combine Datasets

### df_installments_payments

In [119]:
df_installments_payments_train = df_installments_payments[df_installments_payments["SK_ID_CURR"].isin(df_application_train["SK_ID_CURR"])]
df_installments_payments_test = df_installments_payments[df_installments_payments["SK_ID_CURR"].isin(df_application_test["SK_ID_CURR"])]

In [120]:
df_application_train = pd.merge(df_application_train, df_installments_payments_train, on="SK_ID_CURR", how="outer", suffixes=('_application', '_installments_payments'))
df_application_test = pd.merge(df_application_test, df_installments_payments_test, on="SK_ID_CURR", how="outer", suffixes=('_application', '_installments_payments'))

### df_credit_card_balance

In [121]:
df_credit_card_balance_train = df_credit_card_balance[df_credit_card_balance["SK_ID_CURR"].isin(df_application_train["SK_ID_CURR"])]
df_credit_card_balance_test = df_credit_card_balance[df_credit_card_balance["SK_ID_CURR"].isin(df_application_test["SK_ID_CURR"])]

In [122]:
df_application_train = pd.merge(df_application_train, df_credit_card_balance_train, on="SK_ID_CURR", how="outer", suffixes=('_application', '_credit_card_balance'))
df_application_test = pd.merge(df_application_test, df_credit_card_balance_test, on="SK_ID_CURR", how="outer", suffixes=('_application', '_credit_card_balance'))

### df_pos_cash_balance

In [123]:
df_pos_cash_balance_train = df_pos_cash_balance[df_pos_cash_balance["SK_ID_CURR"].isin(df_application_train["SK_ID_CURR"])]
df_pos_cash_balance_test = df_pos_cash_balance[df_pos_cash_balance["SK_ID_CURR"].isin(df_application_test["SK_ID_CURR"])]

In [124]:
df_application_train = pd.merge(df_application_train, df_pos_cash_balance_train, on="SK_ID_CURR", how="outer", suffixes=('_application', '_pos_cash_balance'))
df_application_test = pd.merge(df_application_test, df_pos_cash_balance_test, on="SK_ID_CURR", how="outer", suffixes=('_application', '_pos_cash_balance'))

### df_previous_application

In [125]:
df_previous_application_train = df_previous_application[df_previous_application["SK_ID_CURR"].isin(df_application_train["SK_ID_CURR"])]
df_previous_application_test = df_previous_application[df_previous_application["SK_ID_CURR"].isin(df_application_test["SK_ID_CURR"])]

In [126]:
df_application_train = pd.merge(df_application_train, df_previous_application_train, on="SK_ID_CURR", how="outer", suffixes=('_application', '_previous_application'))
df_application_test = pd.merge(df_application_test, df_previous_application_test, on="SK_ID_CURR", how="outer", suffixes=('_application', '_previous_application'))

### df_bureau_balance and df_bureau

In [127]:
df_bureau_train = df_bureau[df_bureau["SK_ID_CURR"].isin(df_application_train["SK_ID_CURR"])]
df_bureau_test = df_bureau[df_bureau["SK_ID_CURR"].isin(df_application_test["SK_ID_CURR"])]

In [128]:
df_application_train = pd.merge(df_application_train, df_bureau_train, on="SK_ID_CURR", how="outer", suffixes=('_application', '_bureau'))
df_application_test = pd.merge(df_application_test, df_bureau_test, on="SK_ID_CURR", how="outer", suffixes=('_application', '_bureau'))

In [129]:
gc.collect()

252

# Model Building

## Train-Validation Split

In [130]:
input_columns = df_application_train.columns
input_columns = input_columns[input_columns != 'TARGET']
target_column = 'TARGET'

X = df_application_train[input_columns]
y = df_application_train[target_column]
gc.collect()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.7)

c:\users\karti\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [131]:
# num_train, num_feature = X_train.shape

In [132]:
# xgb_train = xgb.DMatrix(X_train, y_train)
# xgb_eval  = xgb.DMatrix(X_test, y_test)

In [133]:
# xgb_test   = xgb.DMatrix(df_application_test)

In [134]:
# # #Params for the Hyperopt algo
# N_HYPEROPT_PROBES = 8 # #Number of evaluation cycles
# HYPEROPT_ALGO = tpe.suggest  # #Tree-of-Parzen-Estimators algo

# # #Params for XGBoost CV
# NUM_BOOST_ROUNDS = 270
# NB_CV_FOLDS = 10
# EARLY_STOPPING = 200
# HOLDOUT_SIZE = 0.20
# # HOLDOUT_SEED = 123456
# # SEED0 = random.randint(1,1000000000)

In [135]:
# obj_call_count = 0
# cur_best_score = 0

In [136]:
# def objective(space):
#     # #Global Variable Definition
#     global obj_call_count, cur_best_score, X_train, y_train, test, X_val, y_val

#     obj_call_count += 1
#     print('\nXGBoost objective call #{} cur_best_score={:7.5f}'.format(obj_call_count,cur_best_score))

#     sorted_params = sorted(space.items(), key=lambda z: z[0])
#     print('Params:', str.join(' ', ['{}={}'.format(k, v) for k, v in sorted_params if not k.startswith('column:')]))


#     xgb_params = sample(space)
#     model = xgb.cv(xgb_params, xgb_train,
#                    num_boost_round = NUM_BOOST_ROUNDS,
#                     nfold=NB_CV_FOLDS,
#                     stratified=False,
#                     early_stopping_rounds=EARLY_STOPPING,
#                     verbose_eval=100,
#                     show_stdv=False)

#     n_rounds = len(model["test-auc-mean"])
#     cv_score = model["test-auc-mean"][n_rounds-1]
#     print('CV finished n_rounds={} cv_score={:7.5f}'.format(n_rounds, cv_score ))
    
#     xgb_model = xgb.train(
#                         xgb_params,
#                         xgb_train,
#                         num_boost_round=n_rounds,
#                         verbose_eval=True)
    
#     predictions = xgb_model.predict(xgb_eval, ntree_limit =n_rounds)
#     score = roc_auc_score(y_test, predictions)
#     print('valid score={}'.format(score))
    
#     if score > cur_best_score:
#         cur_best_score = score
#         print('NEW BEST SCORE={}'.format(cur_best_score))
       
#     loss = 1 - score
#     return {'loss': loss, 'status': STATUS_OK}

In [137]:
# # #NOTE: Any change in `space`, needs to be changed in xgb_default_params as well
# space ={
#     'booster '    : 'gbtree',       
#     'objective'   : 'binary:logistic',
#     'eval_metric' : 'auc',
#     'seed'        : 42,
#     'silent'      : 0,      #Messages would be printed
#     'n_thread'    : -1,     #-1: all cores are used
#     'subsample'   : 0.8,
#     'colsample_bytree': 0.7,
    
#     'eta'         : hp.uniform('eta', 0.025, 0.25),   # #Learning rate - Step size shrinkage to handle overfitting
#     'min_child_weight': hp.choice("min_child_weight", np.arange(5, 15,dtype=int)), # #Tradeoff b/n over and underfitting
#     'max_depth'   : hp.choice("max_depth", np.arange(4, 8,dtype=int)), # #Tradeoff b/n over and underfitting
#     'alpha'       : hp.uniform('alpha', 0.5, 5), # #L1 regularization term - increase this value will make model more conservative.
#     'lambda'      : hp.uniform('lambda', 0.5, 5), # #L2 regularization term - increase this value will make model more conservative.
#     'gamma'       : hp.uniform('gamma', 0.6, 0.8),
#    }

In [138]:
# # #Trials keep track of all the experiments
# trials = Trials()

# # #MAIN function to run all the experiments
# best = fmin(fn=objective,
#                      space=space,
#                      algo=HYPEROPT_ALGO,
#                      max_evals=N_HYPEROPT_PROBES,
#                      trials=trials,
#                      verbose=1)


# print('-'*50)
# print('The best params:')
# print( best )
# print('\n\n')

In [139]:
# best

In [140]:
# # #NOTE: Any change in xgb_default_params, needs to be changed in `space` as well
# xgb_default_params ={
#     'booster '    : 'gbtree',       
#     'objective'   : 'binary:logistic',
#     'eval_metric' : 'auc',
#     'seed'        : 42,
#     'silent'      : 0,      #Messages would be printed
#     'n_thread'    : -1,     #-1: all cores are used
#     'subsample'   : 0.8,
#     'colsample_bytree': 0.7,
# }

In [141]:
# xgb_params = {**xgb_default_params, **best}

In [142]:
# # xgb_params
# {'alpha': 2.7266306946310426,
#  'booster ': 'gbtree',
#  'colsample_bytree': 0.7,
#  'eta': 0.11084829740057514,
#  'eval_metric': 'auc',
#  'gamma': 0.6217758787178843,
#  'lambda': 2.9036392904139543,
#  'max_depth': 0,
#  'min_child_weight': 4,
#  'n_thread': -1,
#  'objective': 'binary:logistic',
#  'seed': 42,
#  'silent': 0,
#  'subsample': 0.8}

In [143]:
# #LB: 0.779
xgb_params = {
 'alpha': 3.160842634951819,
 'booster ': 'gbtree',
 'colsample_bytree': 0.7,
 'eta': 0.1604387053222455,
 'eval_metric': 'auc',
 'gamma': 0.6236454630290655,
 'lambda': 4.438488456929287, 
 'max_depth': 4,
 'min_child_weight': 9,
 'n_thread': -1,
 'objective': 'binary:logistic',
 'seed': 42,
 'silent': 0,
 'subsample': 0.8
}

In [144]:
# #Final Model
gc.collect()
watchlist = [(xgb.DMatrix(X_train, y_train), 'train'), (xgb.DMatrix(X_test, y_test), 'valid')]
model = xgb.train(xgb_params, xgb.DMatrix(X, y), 270, watchlist, maximize=True, verbose_eval=100)

[0]	train-auc:0.700773	valid-auc:0.699844
[100]	train-auc:0.803253	valid-auc:0.800399
[200]	train-auc:0.823596	valid-auc:0.820257
[269]	train-auc:0.833834	valid-auc:0.831142


In [145]:
df_predict = model.predict(xgb.DMatrix(df_application_test), ntree_limit=model.best_ntree_limit)

In [146]:
submission = pd.DataFrame()
submission["SK_ID_CURR"] =  df_application_test["SK_ID_CURR"]
submission["TARGET"] =  df_predict

submission.to_csv("../submissions/model_1_xgbstarter_updatedParams_v10.csv", index=False)

In [147]:
# #Should be 48744, 2
submission.shape

(57692, 2)

In [148]:
# #Feature Importance
sorted(((value,key) for (key,value) in model.get_fscore().items()), reverse=True)
# importance = model.get_fscore()

[(137, 'K_APP_ANNUITY_TO_CREDIT_RATIO'),
 (124, 'EXT_SOURCE_3'),
 (120, 'EXT_SOURCE_2'),
 (117, 'EXT_SOURCE_1'),
 (94, 'DAYS_BIRTH'),
 (80, 'K_INST_AMT_DIFF'),
 (59, 'K_BUREAU_DAYS_CREDIT_MAX'),
 (59, 'K_BUREAU_DAYS_CREDIT_ENDDATE_MAX'),
 (55, 'K_INST_DAYS_DIFF'),
 (55, 'AMT_ANNUITY'),
 (54, 'K_CNT_INSTALMENT_FUTURE_MEAN'),
 (52, 'K_APP_GOODSPRICE_TO_CREDIT_RATIO'),
 (49, 'K_BUREAU_AMT_CREDIT_SUM_MAX'),
 (47, 'K_DAYS_BIRTH_TO_EMPLOYED_RATIO'),
 (47, 'K_BUREAU_AMT_CREDIT_SUM_DEBT_MEAN'),
 (46, 'DAYS_ID_PUBLISH'),
 (45, 'K_AMT_ANNUITY_MEAN'),
 (43, 'K_BUREAU_DAYS_CREDIT_SORTED_SUCCESSIVE_DIFF_MEAN'),
 (42, 'K_APP_ANNUITY_TO_INCOME_RATIO'),
 (41, 'K_CNT_DRAWINGS_CURRENT_MEAN'),
 (40, 'K_CREDIT_ALLOCATED_MEAN'),
 (40, 'K_BUREAU_AMT_CREDIT_SUM_MEAN'),
 (40, 'DAYS_LAST_PHONE_CHANGE'),
 (39, 'K_BUREAU_DAYS_CREDIT_MEAN'),
 (38, 'K_BUREAU_AMT_CREDIT_SUM_MIN'),
 (37, 'K_BUREAU_DAYS_ENDDATE_FACT_MAX'),
 (37, 'K_BUREAU_AMT_CREDIT_MAX_OVERDUE_MEAN'),
 (36, 'K_PREV_POS_CASH_BALANCE_COUNT'),
 (36, 'K

In [149]:
# #Total Number of Features --- 189
len(sorted(((value,key) for (key,value) in model.get_fscore().items()), reverse=True))

195

In [150]:
# #Number of Features > 50 --- 12
len(sorted(((value,key) for (key,value) in model.get_fscore().items() if value > 50), reverse=True))

12